In [1]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from PathFinder import find_path

In [2]:
#read data
PATH = find_path()

#select files
#files = ['application_train.csv','application_test.csv',]
files = ['preprocessed_train.csv', 'preprocessed_test.csv']
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0,engine='python')
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

files used: ['preprocessed_train.csv', 'preprocessed_test.csv']


In [3]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #235 cols
print('Testing Features shape: ', app_test.shape) #235 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [4]:
#also please note that this is not how you should do this kind of baseline checking
#its just the fastest way I could think of

#apply logistic regression
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001, random_state=20)

# Train on the training data
log_reg.fit(app_train_x, app_train_y)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=20,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [5]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(app_test)[:, 1]

In [11]:
#prediction = 0.0
#prediction = 0.5
prediction = 1.0

# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = (log_reg_pred * 0.0) + prediction

submit.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,SK_ID_CURR,TARGET
0,100001,1.0
1,100005,1.0
2,100013,1.0
3,100028,1.0
4,100038,1.0


In [12]:
# Save the submission to a csv file
overwrite = True
if overwrite:
    #submit.to_csv('results/all_zeros.csv', index = False)
    #submit.to_csv('results/all_point_fives.csv', index = False)
    submit.to_csv('results/all_ones.csv', index = False)

Submitted to kaggle  
all zeros score: 0.5  
all point fives score: 0.5  
all ones score: 0.5  